In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, regexp_replace, col, when, sum

In [4]:
spark = SparkSession.builder.appName("Cleaning_events").getOrCreate()

In [ ]:
# Data is storred in "Dummies" way, therefore it needs to be reorganized
# 1- Select and Keep Only necessary columns
# 2- Remove irrelevant data
# 3- Create new columns to store kept data (or probably new files by splitting the data by type)

In [ ]:
# We also need to create
# 'distance to goal',
# 'Shot angle',
# 'number of players in area of shot',
# 'number of close teammates',
# 'fav foot' : most passes/shots with that foot

In [5]:
events = spark.read.csv("hdfs://localhost:9000/big-data/events.csv",header=True)

In [21]:
events.count()

8710714

In [8]:
events.printSchema()

root
 |-- 50_50: string (nullable = true)
 |-- bad_behaviour_card: string (nullable = true)
 |-- ball_receipt_outcome: string (nullable = true)
 |-- ball_recovery_offensive: string (nullable = true)
 |-- ball_recovery_recovery_failure: string (nullable = true)
 |-- block_deflection: string (nullable = true)
 |-- block_offensive: string (nullable = true)
 |-- carry_end_location: string (nullable = true)
 |-- clearance_aerial_won: string (nullable = true)
 |-- clearance_body_part: string (nullable = true)
 |-- clearance_head: string (nullable = true)
 |-- clearance_left_foot: string (nullable = true)
 |-- clearance_right_foot: string (nullable = true)
 |-- counterpress: string (nullable = true)
 |-- dribble_nutmeg: string (nullable = true)
 |-- dribble_outcome: string (nullable = true)
 |-- dribble_overrun: string (nullable = true)
 |-- duel_outcome: string (nullable = true)
 |-- duel_type: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- foul_committed_advantage: st

In [ ]:
events.createOrReplaceTempView("events")

result_df = spark.sql("SELECT count(*) FROM events WHERE type = 'Shot'")

result_df.show()

+--------+
|count(1)|
+--------+
|   12005|
+--------+



In [51]:
# Trying with Goalkeeper
constant_f = ['id','type']

features = ['goalkeeper_body_part','goalkeeper_end_location','goalkeeper_outcome','goalkeeper_position','goalkeeper_technique',
          'goalkeeper_type']


conditions = ' IS NOT NULL OR '.join(features) + ' IS NOT NULL'

query = f'''
SELECT {','.join(constant_f)},{','.join(features)} FROM events

WHERE {conditions};
'''

rs = spark.sql(query)
rs.show()

+--------------------+-----------+--------------------+-----------------------+------------------+-------------------+--------------------+---------------+
|                  id|       type|goalkeeper_body_part|goalkeeper_end_location|goalkeeper_outcome|goalkeeper_position|goalkeeper_technique|goalkeeper_type|
+--------------------+-----------+--------------------+-----------------------+------------------+-------------------+--------------------+---------------+
|07d5688a-516d-4ad...|Goal Keeper|                NULL|            [1.3, 38.2]|              NULL|                Set|                NULL|     Shot Faced|
|a0a59d50-4998-4ec...|Goal Keeper|                NULL|                   NULL|             Claim|               NULL|                NULL| Keeper Sweeper|
|927178da-7ce7-4cc...|Goal Keeper|          Both Hands|                   NULL|           Success|                Set|            Standing|     Shot Saved|
|c4bf223a-92ff-4ca...|Goal Keeper|                NULL|         

In [4]:
spark.stop()